# Microsoft Malware Protection Engine Crash
This rule detects a suspicious crash of the Microsoft Malware Protection Engine

## Rule Content
```
- title: Microsoft Malware Protection Engine Crash
  id: 6c82cf5c-090d-4d57-9188-533577631108
  description: This rule detects a suspicious crash of the Microsoft Malware Protection
    Engine
  tags:
  - attack.defense_evasion
  - attack.t1089
  - attack.t1211
  status: experimental
  date: 2017/05/09
  references:
  - https://bugs.chromium.org/p/project-zero/issues/detail?id=1252&desc=5
  - https://technet.microsoft.com/en-us/library/security/4022344
  author: Florian Roth
  logsource:
    product: windows
    service: application
    category: null
  detection:
    selection1:
      Source: Application Error
      EventID: 1000
    selection2:
      Source: Windows Error Reporting
      EventID: 1001
    keywords:
      Message:
      - '*MsMpEng.exe*'
      - '*mpengine.dll*'
    condition: 1 of selection* and all of keywords
  falsepositives:
  - MsMpEng.exe can crash when C:\ is full
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-application-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(((source_name:"Application\ Error" AND event_id:"1000") OR (source_name:"Windows\ Error\ Reporting" AND event_id:"1001")) AND Message.keyword:(*MsMpEng.exe* OR *mpengine.dll*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()